In [75]:
import numpy as np
import pandas as pd

file_path = "../../data/processed/hi_rws_0001_0256_descriptive.csv"
df = pd.read_csv(file_path, nrows=100, memory_map=True)

## Previous Preprocessing Steps

1. Sent Tokens
2. Word Tokens
3. Remove punctuations 
4. Remove stopwords (NLTK List)
5. Lower words
6. Lemmatize
7. Stemming

In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 21 columns):
alias               100 non-null object
ratingValue         100 non-null int64
dataPublished       100 non-null object
description         100 non-null object
author              100 non-null object
sentiment           100 non-null int64
word_count          100 non-null int64
sent_count          100 non-null int64
chr_count           100 non-null int64
avg_word_len        100 non-null float64
avg_sent_len        100 non-null float64
num_of_stopwords    100 non-null int64
num_of_modals       100 non-null int64
hashtags            100 non-null int64
mentions            100 non-null int64
numerics            100 non-null int64
uppercase_cnt       100 non-null int64
punctuation_cnt     100 non-null int64
vocab_cnt           100 non-null int64
ratio_lexical       100 non-null float64
ratio_content       100 non-null float64
dtypes: float64(4), int64(13), object(4)
memory usage: 16.5+ KB


In [77]:
import spacy
nlp = spacy.load("en_core_web_sm", disable=['ner'])
print(spacy.__version__)
nlp.pipe_names
nlp

2.0.18


In [217]:
text = nlp(df.description[5])
sent = [i for i in text.sents][0]
print(dir(text))
type(text[0].lemma_)

['_', '__bytes__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__ne__', '__new__', '__pyx_vtable__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__unicode__', '_bulk_merge', '_py_tokens', '_realloc', '_vector', '_vector_norm', 'cats', 'char_span', 'count_by', 'doc', 'ents', 'extend_tensor', 'from_array', 'from_bytes', 'from_disk', 'get_extension', 'get_lca_matrix', 'has_extension', 'has_vector', 'is_parsed', 'is_sentenced', 'is_tagged', 'mem', 'merge', 'noun_chunks', 'noun_chunks_iterator', 'print_tree', 'remove_extension', 'retokenize', 'sentiment', 'sents', 'set_extension', 'similarity', 'tensor', 'text', 'text_with_ws', 'to_array', 'to_bytes', 'to_disk', 'user_data', 'user_hooks', 'user_span_hooks', 'user_token_hooks', 'vector', 'vector_

str

In [218]:
[i for i in text.noun_chunks]
text[0].sent.text != 'I stumbled across this great my vacation to Maui.'

True

In [ ]:
# solution one that I dont like
texts = []
for sent_token in text.sents:

    for token in text:        
        if token in sent_token:
            # do cleaning in here
            texts.append(clean_up(sent_token.text))
[tuple(i) for i in texts]

In [220]:
# solution that is fair enough
texts = []
sent_current = ""
for token in text:     
    # check for tokens current sent
    if sent_current == token.sent.text: 
        # add same sent tokens to the sent list
        token_clean = token_clean_up(token)
        if token_clean is not None:
            sent.append(token_clean)
            print(sent)
    else:         
        # add it to texts, if it is not initially
        if sent_current != "":
            texts.append(sent)
        # update current sent index
        sent_current = token.sent.text
        # create sent list and add first token
        sent = []
        token_clean = token_clean_up(token)
        if token_clean is not None:
            sent.append(token_clean)
texts.append(sent)
print(len(list(text.sents)), len(texts))
# [print(i) for i in text.sents]
# [tuple(i) for i in texts]
# [print(i, '\n\n', y) for i,y in zip(text.sents, texts)]

['what', 'amazing']
['what', 'amazing', 'restaurant']
['what', 'amazing', 'restaurant', 'view']
['what', 'amazing', 'restaurant', 'view', 'dinner']
['din']
['din', 'wave']
['din', 'wave', 'crash']
['din', 'wave', 'crash', 'nice']
['din', 'wave', 'crash', 'nice', 'relaxing']
['food']
['food', 'good']
['food', 'good', 'start']
['food', 'good', 'start', 'avocado']
['food', 'good', 'start', 'avocado', 'stack']
['food', 'good', 'start', 'avocado', 'stack', 'disappoint']
['order']
['order', 'steak']
['order', 'steak', 'lobster']
['order', 'steak', 'lobster', 'notice']
['order', 'steak', 'lobster', 'notice', 'lobster']
['order', 'steak', 'lobster', 'notice', 'lobster', 'tail']
['order', 'steak', 'lobster', 'notice', 'lobster', 'tail', 'seasoning']
['order', 'steak', 'lobster', 'notice', 'lobster', 'tail', 'seasoning', 'come']
['order', 'steak', 'lobster', 'notice', 'lobster', 'tail', 'seasoning', 'come', 'char']
['order', 'steak', 'lobster', 'notice', 'lobster', 'tail', 'seasoning', 'come', '

In [135]:
def token_clean_up(token):
    """ token cleanup. Return clean token or None. """
    removal=['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE']
    if token.is_stop == False and token.is_alpha and len(token)>3 and token.pos_ not in removal:
        lemma = token.lemma_
        return lemma

In [110]:
def clean_up(text):  
    """ clean up tokens by documents """
    removal = ['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE']
    doc = nlp(text)
    text_out = []    
    for token in doc:
        if token.is_stop == False and token.is_alpha and len(token)>3 and token.pos_ not in removal:
            lemma = token.lemma_
            text_out.append(lemma)
    return text_out

In [150]:
def clean_up2(text, clean_up=False):
    """ clean up tokens by sents in documents """
    removal=['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE']
    doc = nlp(text)
    
    texts = []
    sent_current = ""
    for token in doc:    
        # check for tokens current sent
        if sent_current != token.sent.text:
            # add it to texts, if it is not initially
            if sent_current != "":
                texts.append(sent)
            # update current sent index
            sent_current = token.sent.text
            # create sent list and add first token
            sent = []
            if clean_up:
                token_clean = token_clean_up(token)
                if token_clean is not None:
                    sent.append(token_clean)
            else:
                sent.append(token)
        else:        
            # add same sent tokens to the sent list
            if clean_up:
                token_clean = token_clean_up(token)
                if token_clean is not None:
                    sent.append(token_clean)
            else:
                sent.append(token)
    return texts

In [191]:
df_test = df.description.apply(lambda x: clean_up2(x, True))
type(df_test[0][0][4])

str

In [192]:
# testing with gensim
from itertools import chain
from gensim.corpora import Dictionary

dictionary = Dictionary(chain(*df_test))
len(dictionary)

1127